# Model Parser Example

Demo of how to query for an existing model that is already in a database

In [1]:
# Specify the CIM profile version, using GridAPPS-D extensions, ver. 2021
import importlib
cim_profile = 'rc4_2021'
cim = importlib.import_module('cimgraph.data_profile.' + cim_profile)
sparql = importlib.import_module('cimgraph.loaders.sparql.' + cim_profile)

In [2]:
# Import Distributed Topology Processor
from topology_processor import DistributedTopology

In [3]:
# Import CIMantic Graphs database connection and models
from cimgraph.loaders import Parameter, ConnectionParameters
from cimgraph.loaders.blazegraph.blazegraph import BlazegraphConnection
from cimgraph.models import DistributedModel
import json

In [4]:
# Connect to Blazegraph Database
params = ConnectionParameters([Parameter(key="url", value="http://localhost:8889/bigdata/namespace/kb/sparql")])
bg = BlazegraphConnection(params, 'rc4_2021')

In [5]:
# Create Feeder object
feeder_mrid = "_49AD8E07-3BF9-A4E2-CB8F-C3722F837B62"  # 13 bus
feeder = cim.Feeder(mRID=feeder_mrid)

In [6]:
# Run Distributed Topology Processor to create switch-delimited topological areas
Topology = DistributedTopology(bg, feeder_mrid)
topology_response = json.loads(Topology.create_switch_areas(feeder_mrid))

In [7]:
# Create CIMantic Graphs model and initialize all CIM objects
network = DistributedModel(connection=bg, feeder=feeder, topology=topology_response['feeders'])

In [8]:
network.pprint(cim.ACLineSegment)

AttributeError: 'DistributedModel' object has no attribute 'pprint'